# Functions for Datasets Provided by Lex's R Code (aka Kickbacks)
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** November 17, 2021

***
`This Jupyter Notebook contains five functions that are used when exploring the records that have been returned by Lex's R-Code (aka Kickback). It should be noted that the main dataframe provided by the kickback must have a field named 'UASLOCATION'.`
***
The five functions contained within this notebook are as follows:
1. uas_loc_parts()
2. uas_loc_partsCITY()
3. link_airportsCleaned_ident()
4. link_airportsCleaned_icao()
5. nav_link()
6. uas_lat_long()

***

### <font color='blue'> uas_loc_parts()</font>

In [1]:
def uas_loc_parts(dataframe):
    '''
    This function takes the 'UASLOCATION' column from Lex's 'Kickbacks' and breaks it into the different parts of the 
    UAS Location, to include: Distance (NM), Direction Abbreviation, Airport Identifier
    
    input : dataframe is a df that includes the 'UASLOCATION' field
    output : the dataframe provided as input to the user with additional fields to include: 'Distance_NM', 'Bearing', 
                                                                                            'IDENT'
    '''
    uasLoc_col = dataframe['UASLOCATION']
    dist_list = []
    bearing_list = []
    ident_list = []
    dist_kilo = []
    bearDegree = []
    
    # Dictionary of Bearing Abbreviation to Degrees Conversions
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    beardeg_list = list(bearing_deg.keys())
    
    for i in range(len(dataframe)):
        # Get Distance (NM) from UAS LOCATION and Convert Distance to Kilometers
        distNM = re.sub("[^\d\.]","", str(re.findall(r"[-+]?\d*\.\d+|\d+", uasLoc_col[i].split()[0])))
        if distNM == '':
            dist_list.append(0)
            dist_kilo.append(0.00)
        else:
            dist_list.append(distNM)
            dist_kilo.append(float(distNM) * 1.852)
        
        # Get IDENT from UASLOCATION
        ident = uasLoc_col[i].split()[-1].replace('of', '')
        ident_list.append(ident)
        
        # Get Bearing from UASLOCATION and Convert Bearing to Degrees
        #bearing = uasLoc_col[i].replace('NM', '').replace('of','').split()[-2]
        bearing = re.sub('(NM|of|OF)', ' ', uasLoc_col[i]).split()[-2]
        bearing_list.append(bearing)
        if bearing.isdigit():
            bearDegree.append(bearing)
        else:
            bearDegree.append(bearing_deg.get(bearing))
        
    # Append new fields to dataframe provided as input    
    dataframe['Distance_NM'] = dist_list
    dataframe['Distance_Kilometers'] = dist_kilo
    dataframe['Bearing'] = bearing_list
    dataframe['Bearing_Degrees'] = bearDegree
    dataframe['IDENT'] = ident_list
    
    
    return dataframe

### <font color='blue'> uas_loc_partsCITY()</font>

In [1]:
def uas_loc_partsCITY(dataframe):
    '''
    This function takes the 'UASLOCATION' column from Lex's 'Kickbacks' and breaks it into the different parts of the 
    UAS Location, to include: Distance (NM), Direction Abbreviation
    
    input : dataframe is a df that includes the 'UASLOCATION' field and a 'cityname' field
    output : the dataframe provided as input to the user with additional fields to include: 'Distance_NM', 'Bearing'
    '''
    uasLoc_col = dataframe['UASLOCATION']
    dist_list = []
    bearing_list = []
    city_list = []
    dist_kilo = []
    bearDegree = []
    
    # Dictionary of Bearing Abbreviation to Degrees Conversions
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    beardeg_list = list(bearing_deg.keys())
    
    for i in range(len(dataframe)):
        # Get Distance (NM) from UAS LOCATION and Convert Distance to Kilometers
        distNM = re.sub("[^\d\.]","", str(re.findall(r"[-+]?\d*\.\d+|\d+", uasLoc_col[i].split()[0])))
        if distNM == '':
            dist_list.append(0)
            dist_kilo.append(0.00)
        else:
            dist_list.append(distNM)
            dist_kilo.append(float(distNM) * 1.852)
        
        # Get CITY uppercase
        dataframe['cityname'].loc[i] = dataframe['cityname'][i].upper()
        
        # Get Bearing from UASLOCATION and Convert Bearing to Degrees
        #bearing = uasLoc_col[i].replace('NM', '').replace('of','').split()[-2]
        
        bearing = uasLoc_col[i].split()[-1]
        bearing_list.append(bearing)
        if bearing.isdigit():
            bearDegree.append(bearing)
        else:
            bearDegree.append(bearing_deg.get(bearing))
        
    # Append new fields to dataframe provided as input    
    dataframe['Distance_NM'] = dist_list
    dataframe['Distance_Kilometers'] = dist_kilo
    dataframe['Bearing'] = bearing_list
    dataframe['Bearing_Degrees'] = bearDegree
    dataframe['NAME_TXT'] = dataframe['cityname']
    
    
    return dataframe

### <font color='blue'> link_airportsCleaned_ident()

In [2]:
def link_airportsCleaned_ident(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='IDENT', how='left')

### <font color='blue'> link_airportsCleaned_icao()

In [4]:
def link_airportsCleaned_icao(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='ICAO_ID', how='left')

### <font color='blue'> nav_link()

In [5]:
def nav_link (dataframe):
    '''
    This functions merges the dataframe input by the user with the cleaned NAVAID_cleaned.csv dataset
    '''
    
    # Read in Cleaned NAVAID Systems dataset
    nav_clean = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/NAVAID_cleaned.csv')
    
    output_df = pd.merge(dataframe, nav_clean, on = 'IDENT', how = 'left').reset_index(drop=True)
    
    return output_df

### <font color='blue'> uas_lat_long()

In [3]:
def uas_lat_long(dataframe):  
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the bearing_dist_originIdent() function to output the dataframe containing complete UAS Location information
    in terms of the UAS' Latitude and Longitude.
    
    @param dataframe : The dataframe that includes the fields 'LATITUDE', 'LONGITUDE', 'Bearing_Degrees', 
                        'Distance_Kilometers' to have UAS Lat/Long values calculated for
    
    @output new_df: returns a dataframe containing the Remark, UAS Location, FAA Airport Identifier, Distance (in NM) from 
                    the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks for which UAS latitudes and longitudes
                    were calculated is located in the StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv file.
                    
    
    '''
    uas_lat = []
    uas_long = []
    
    for i in range(len(dataframe)):
        lat_airport = pd.to_numeric(dataframe['LATITUDE'][i])
        long_airport = pd.to_numeric(dataframe['LONGITUDE'][i])
        b = pd.to_numeric(dataframe['Bearing_Degrees'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_airport, long_airport)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2 = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)
        
    
    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    #dataframe.to_csv('StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv', index = False)
    
    return dataframe